In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import time
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import argparse
from pmlb import fetch_data
from sklearn import datasets

In [2]:
test = True
# test = False
n_splits = 5

In [3]:
# データセットの読み込み
# fetch data のリスト
reviewed_datasets = [
     'molecular_biology_promoters',
     'car',
     'connect_4',
     'dna',
     '542_pollution',
     '560_bodyfat',
     '1089_USCrime',
     '529_pollen',
     'chess',
     'penguins',
     'bupa',
     'movement_libras',
     'adult',
     'waveform_21',
     'waveform_40',
     'saheart',
     'wine_quality_white',
     'wine_quality_red',
     'irish',
     'mushroom'
 ]

n_splits = 5  # kfoldでcross validationする回数

if test:
    data_kind_list = ["B","M","R"]
    use_data_name = [reviewed_datasets[0], reviewed_datasets[1], reviewed_datasets[4]]
    dfsets = list(map(fetch_data, use_data_name))
else:
    sk_datasets_name = ["load_boston", "load_digits", "load_wine", "fetch_california_housing"]
    sk_datasets = []
    for name in sk_datasets_name:
        print(name)
        data = eval("datasets.{}()".format(name))
        X = pd.DataFrame(data.data, columns=data.feature_names)
        y = pd.DataFrame(data.target,columns=["target"])
        df = pd.concat([X,y], axis=1)
        sk_datasets.append(df)
    use_data_name = reviewed_datasets
    dfsets = list(map(fetch_data, use_data_name)) + sk_datasets
    use_data_name = use_data_name + sk_datasets_name
    # "B": binary classifier, "C": multi classifier, "R":regression
    data_kind_list = ["B", "M", "B", "B", "R","R","R","R", "B", "B", "B","M","B","M","M","B","M","M","B","B","R","R","R","R"]


In [4]:
adult = fetch_data("adult")

In [68]:
num_rows = adult.shape[0]
train = adult.iloc[:round(num_rows*0.8), :]
test = adult.iloc[round(num_rows*0.8):, :]

In [6]:
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]

NameError: name 'train' is not defined

In [77]:
X_test = test.iloc[:, :-1]
y_test = test.loc[:, "target"]

In [76]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

In [80]:
y_pred_gbc = gbc.predict(X_test)

In [105]:
accuracy_score(y_test, y_pred_gbc)

ValueError: Found input variables with inconsistent numbers of samples: [9769, 9768]

In [7]:
kf = KFold(n_splits=5, shuffle=True)
a,b,c,d,e = kf.split(adult)
train_row, test_row = a

In [8]:
train = adult.iloc[train_row, :]
test = adult.iloc[test_row, :]

X_train = train.iloc[:,:-1]  # target以外の列
y_train  = train.iloc[:,-1]  # targetの行
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

In [10]:
adult

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0.0,40.0,39,1
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0.0,13.0,39,1
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0.0,40.0,39,1
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0.0,40.0,39,1
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0.0,40.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39.0,4,215419.0,9,13.0,0,10,1,4,0,0.0,0.0,36.0,39,1
48838,64.0,0,321403.0,11,9.0,6,0,2,2,1,0.0,0.0,40.0,39,1
48839,38.0,4,374983.0,9,13.0,2,10,0,4,1,0.0,0.0,50.0,39,1
48840,44.0,4,83891.0,9,13.0,0,1,3,1,1,5455.0,0.0,40.0,39,1


In [11]:
train_left_columns = ["age", "sex", "workclass", "fnlwgt", "education"]
train_right_columns = ["age", "sex"] + list( set(X_train.columns) - set(train_left_columns) )

train_left = X_train.loc[:, train_left_columns]
train_right = X_train.loc[:, train_right_columns]

In [41]:
pipe_lr = Pipeline([("scl",StandardScaler()),("est",LogisticRegression()) ])
pipe_gb = Pipeline([("scl",StandardScaler()),("est",GradientBoostingClassifier()) ])

In [45]:
pipe_lr.fit(X_train, y_train)
pipe_gb.fit(X_train, y_train)

Pipeline(steps=[('scl', StandardScaler()),
                ('est', GradientBoostingClassifier())])

In [49]:
y_pred_lr = pipe_lr.predict(X_test)
y_pred_gb  =pipe_gb.predict(X_test)

In [51]:
accuracy_score(y_test, y_pred_lr)

0.8244446719213839

In [52]:
accuracy_score(y_test, y_pred_gb)

0.8664141672637936